In [1]:
import dataloader
import pandas as pd
import utils
from model import generate_model
import os
import torch
import torch.nn as nn

from optimizer import Adam,SGD

from train_wrapper import train_epoch
import datetime

In [2]:
config = utils.load_config()

batch_size = config['dataloader']['batch_size']
num_workers = config['dataloader']['num_workers']
pin_memory = config['dataloader']['pin_memory'] == 1 
gpu_parallel = config['gpus']
learning_rate = config['train']['learning_rate']
lr_steps = config['train']['lr_steps']
epochs = config['train']['epochs']

config['save_datetime'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")


In [3]:
#데이터셋 분리(Train, validation, test)
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
df_dataset = df_dataset.dropna().reset_index(drop=True)
df_oasis = df_dataset[df_dataset['source'] == 'OASIS-3']
df_adni = df_dataset[df_dataset['source'] == 'ADNI']

In [4]:
#데이터 분포도 확인 
data_summary=''
raw_ad,raw_cn,raw_mci = pd.get_dummies(df_dataset['group_maxinc']).sum()

raw_total = raw_ad + raw_cn + raw_mci
data_summary = 'Raw Data\n Total: {}    CN : {} ({:.2f}% of total),   MCI: {} ({:.2f}% of total),   AD : {} ({:.2f}% of total)'.format(raw_total, raw_cn,  100 * raw_cn / raw_total, raw_mci, 100 * raw_mci / raw_total, raw_ad,  100 * raw_ad / raw_total)

oasis_ad,oasis_cn,oasis_mci = pd.get_dummies(df_oasis['group_maxinc']).sum()

oasis_total = oasis_ad + oasis_cn + oasis_mci
data_summary += '\nOasis Data\n Total: {}    CN : {} ({:.2f}% of total),   MCI: {} ({:.2f}% of total),   AD : {} ({:.2f}% of total)'.format(oasis_total, oasis_cn,  100 * oasis_cn / oasis_total, oasis_mci, 100 * oasis_mci / oasis_total, oasis_ad,  100 * oasis_ad / oasis_total)

adni_ad,adni_cn,adni_mci = pd.get_dummies(df_adni['group_maxinc']).sum()
adni_total = adni_ad + adni_cn + adni_mci
data_summary += '\nADNI Data\n Total: {}    CN : {} ({:.2f}% of total),   MCI: {} ({:.2f}% of total),   AD : {} ({:.2f}% of total)'.format(adni_total, adni_cn,  100 * adni_cn / adni_total, adni_mci, 100 * adni_mci / adni_total, adni_ad,  100 * adni_ad / adni_total)

print(data_summary)

Raw Data
 Total: 990    CN : 576 (58.18% of total),   MCI: 263 (26.57% of total),   AD : 151 (15.25% of total)
Oasis Data
 Total: 824    CN : 524 (63.59% of total),   MCI: 183 (22.21% of total),   AD : 117 (14.20% of total)
ADNI Data
 Total: 166    CN : 52 (31.33% of total),   MCI: 80 (48.19% of total),   AD : 34 (20.48% of total)


In [5]:
X_train,X_val,y_train,y_val = dataloader.dataset_split(df_oasis,test_size=0.2,shuffle=True,grp=None,seed=1004)
X_test = df_adni.drop(labels='group_maxinc',axis=1)
y_test = df_adni['group_maxinc']

d:\MyGit\BrainMR_MCI\dataloader.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset['grp'] = (df_dataset['source'].str.replace('OASIS-3','1').str.replace('ADNI','2').apply(pd.to_numeric)*1000


In [6]:
import torchio as tio
from torch.utils.data import DataLoader,ConcatDataset,WeightedRandomSampler

class_counts = y_train.value_counts().to_dict() 
num_samples = sum(class_counts.values()) 
labels = y_train.to_list()

#클래스별 가중치 부여
class_weights = {i:num_samples / class_counts[i] for i in class_counts.keys()}

# 해당 데이터의 label에 해당되는 가중치
weights = [class_weights[labels[i]] for i in range(int(num_samples))] #해당 레이블마다의 가중치 비율

sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))

transform = tio.RandomAffine(degrees=(0,0,90)) #이미지 좌우로 랜덤 생성

traindata=dataloader.MRIDataset_class2(X_train,y_train)
#aug_traindata=dataloader.MRIDataset(X_train,y_train,transform)

#train_plus = ConcatDataset([traindata, aug_traindata])

valdata=dataloader.MRIDataset_class2(X_val,y_val)
testdata=dataloader.MRIDataset_class2(X_test,y_test)

shuffle = True
sampler = None

from torch.utils.data import DataLoader
train_dataloader = DataLoader(traindata , batch_size=batch_size, shuffle=shuffle, sampler = sampler
                              ,num_workers=num_workers,pin_memory = pin_memory)
val_dataloader  = DataLoader(valdata, batch_size=batch_size, shuffle=False
                              ,num_workers=num_workers,pin_memory = pin_memory)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

print('train_dataloader : ',len(train_dataloader.dataset))
print('val_dataloader : ',len(val_dataloader.dataset))
print('test_dataloader : ',len(test_dataloader.dataset))

train_dataloader :  659
val_dataloader :  165
test_dataloader :  166


In [7]:
loss_weights = []
for w in class_weights.values():
    loss_weights.append(w)
    
loss_weights = torch.tensor(loss_weights,dtype=torch.float)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
#case 1: epoch=100, add age = 'N',밸런스 조정 ='N',Crop: (가로,세로,깊이), normalize=minmax(5,95)
config['save_datetime'] = datetime.datetime.now().strftime("%Y%m%d_%H%M")

model_name = 'resnet'
model_depth = 18

model, _ = generate_model(model_name=model_name,model_depth = model_depth
                        ,n_classes=2,resnet_shortcut='B',add_last_fc_num=0)
model.to(device)

if len(gpu_parallel) > 1 and torch.cuda.is_available():
    model = nn.DataParallel(model, device_ids = gpu_parallel)
    model.to(device)

optimizer = Adam(model, learning_rate = learning_rate)
criterion_clf = nn.CrossEntropyLoss().to(device)

utils.save_messgage(config
                    , model_name=model_name, model_depth = model_depth, n_classes=3, resnet_shortcut='B'
                    , optimizer = 'Adam', lr=learning_rate, criterion_clf='CrossEntropyLoss'
                    , data_summary=data_summary, sampler= 'N', agumentation='N',add_last_fc_num=0
                    , class_weights =class_weights
                    , message='비고')
train_epoch(device,train_dataloader,val_dataloader,test_dataloader,model,criterion_clf,optimizer,config,epoch = epochs,age_onoff=True,num_classes=2)

d:\MyGit\BrainMR_MCI\models\resnet.py:143: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


train at epoch 0
Epoch: [0][0/165]	 lr: 0.0900000000	Loss : 1.0235	Acc : 0.25000	
Epoch: [0][10/165]	 lr: 0.0900000000	Loss : 7.4284	Acc : 0.54545	
Epoch: [0][20/165]	 lr: 0.0900000000	Loss : 4.2453	Acc : 0.54762	
Epoch: [0][30/165]	 lr: 0.0900000000	Loss : 3.0796	Acc : 0.60484	
Epoch: [0][40/165]	 lr: 0.0900000000	Loss : 2.5063	Acc : 0.59756	
Epoch: [0][50/165]	 lr: 0.0900000000	Loss : 2.1398	Acc : 0.61275	
Epoch: [0][60/165]	 lr: 0.0900000000	Loss : 1.8926	Acc : 0.62295	
Epoch: [0][70/165]	 lr: 0.0900000000	Loss : 1.7269	Acc : 0.61268	
Epoch: [0][80/165]	 lr: 0.0900000000	Loss : 1.5910	Acc : 0.62346	
Epoch: [0][90/165]	 lr: 0.0900000000	Loss : 1.4715	Acc : 0.64286	
Epoch: [0][100/165]	 lr: 0.0900000000	Loss : 1.3796	Acc : 0.65594	
Epoch: [0][110/165]	 lr: 0.0900000000	Loss : 1.3207	Acc : 0.65315	
Epoch: [0][120/165]	 lr: 0.0900000000	Loss : 1.2650	Acc : 0.65496	
Epoch: [0][130/165]	 lr: 0.0900000000	Loss : 1.2231	Acc : 0.64695	
Epoch: [0][140/165]	 lr: 0.0900000000	Loss : 1.1849	Acc 

IndexError: list index out of range